<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toranto City</font></h1>

## Introduction

In this assignment first we have to clean the data obtained from Wikipedia page for the toranto city and using web scraping method. Then convert addresses into their equivalent latitude and longitude values. Use Foursquare API to explore neighborhoods in Toranto City. Get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. Use the *k*-means clustering algorithm to complete this task. Finally, use the Folium library to visualize the neighborhoods in Toranto City and their emerging clusters.

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [11]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Scrap data from Wikipedia page into a DataFrame

Get the data set from the URL : https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M and then using BeautifulSoup package

In [12]:
dataset = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [13]:
from bs4 import BeautifulSoup # library to parse HTML and XML documents

In [14]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(dataset, 'html.parser')

In [15]:
# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

In [16]:
# View the data of the dataframe
toronto_df.head(20)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


### 2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [17]:
indexToDrop = toronto_df[toronto_df['Borough'] == 'Not assigned'].index
toronto_df.drop(indexToDrop , inplace=True)

In [18]:
toronto_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


## 3.More than one neighborhood can exist in one postal code area

In [19]:
toronto_df_group = toronto_df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))

In [20]:
#Replace the slash with coma(,)
toronto_df_group["Neighborhood"] = toronto_df_group["Neighborhood"].replace(to_replace= r'\/', value= ',', regex=True)

In [21]:
toronto_df_group.head(20)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park"
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge"
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff , Cliffside West"


### 4.If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [22]:
toronto_df_group.loc[toronto_df_group['Neighborhood'] =='Not assigned' , 'Neighborhood'] = toronto_df_group['Borough']

In [23]:
toronto_df_group.head(20)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park"
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge"
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff , Cliffside West"


### 5. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [24]:
toronto_df_group.shape

(103, 3)

## Q2.Use the Geocoder package or the csv file to create dataframe with longitude and latitude values

In [25]:
import pandas as pd
geo_coordinates = pd.read_csv('Geospatial_Coordinates.csv')

In [26]:
geo_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [27]:
# Replacing the column name Postal Code to PostalCode
geo_coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)

## Merged the two dataframe to get the desired dataframe with all the details

In [30]:
# merge two table on the column "PostalCode"
toronto_df_new = toronto_df_group.merge(geo_coordinates, on="PostalCode", how="left")
toronto_df_new.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848
